In [5]:
import gym

env = gym.make('CliffWalking-v0')

states = env.observation_space.n
actions = env.action_space.n

In [6]:
import numpy as np

q_table=np.zeros((states,actions))
v_table=np.zeros(states)

alpha=0.6
gamma=0.99

episodes=100000
eps_steps=episodes*2//3
max_eps=1
min_eps=0.1
eps_a=(max_eps-min_eps)/eps_steps

In [7]:
def eps_greedy(current_state, step_no, trainingOn):
    eps = max(min_eps, max_eps - eps_a*step_no)
    if np.random.uniform() < eps and trainingOn:
        action= np.random.randint(0, actions)
    else:
        q_values = q_table[current_state]
        action = np.argmax(q_values)
    return action

In [21]:
def update_q_table(state, action, reward, new_state):
    q_table[state, action] = (1-alpha)*q_table[state, action] + alpha*(reward + gamma*v_table[new_state])
    v_table[state] = max(q_table[state])

In [22]:
def q_learning(episodes, trainingOn):
    culumative_reward = 0
    cumulative_steps = 0
    avg_rewards_list = []

    for e in range(episodes):
        state = env.reset()
        
        episode_reward = 0
        episode_step_no=0
        done=False
        while not done:
            action = eps_greedy(state, cumulative_steps, trainingOn)
            new_state, reward, done, info = env.step(action)

            episode_step_no += 1
            cumulative_steps += 1
            episode_reward += reward

            if trainingOn:
                update_q_table(state, action, reward, new_state)

            state = new_state

        culumative_reward += episode_reward
        avg_rewards_list.append(culumative_reward/(e+1))
    
    average_reward = culumative_reward/episodes
    print("Average reward: ", average_reward)
    return avg_rewards_list


In [23]:
def print_Qtable():
    m, n = q_table.shape

    print("State\t Left \t Down \t Right \t Up")
    print()
    for i in range(m):
        print("State", i)
        for j in range(n):
            print("\t", q_table[i, j], end=" ")
        print('')


In [24]:
avg_list = q_learning(episodes, True)